In [2]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import datetime as dt
import math
from sklearn.metrics import r2_score
from scipy import stats
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [14]:
# get ambient temperature file
df_temp = pd.read_csv('../../data/raw/TEMP_PICKDATA_2020-12-20_sacramento.csv')
df_temp = df_temp[df_temp['start_hour'].notna()]
df_temp['date'] = pd.to_datetime(df_temp['date'])
df_temp = df_temp[['date','start_hour','value','units']]
df_temp['temp_c'] = (df_temp['value']-32)*5/9

# get the coefficients of the temperature profile
df_temp = df_temp[(df_temp['start_hour'].isin([12,13,14,15,16]))&
                      (df_temp['date'].dt.month.isin([4,5]))].reset_index()
df_temp_coef = df_temp.groupby('date').agg({'date':'first'})
df_temp_coef['a']=0
df_temp_coef['b']=0
df_temp_coef['c']=0

def func(x, a, b, c):
    return a*x*x+b*x+c

for i in df_temp.groupby('date').agg({'date':'first'})['date']:
    df = df_temp[df_temp['date']==i]
    x_amb = (df['start_hour']-12)*60
    y_amb = df['temp_c']
    popt, pcov = curve_fit(func, x_amb/10, y_amb)
    df_temp_coef.loc[df_temp_coef['date']==i, 'a'] = popt[0]
    df_temp_coef.loc[df_temp_coef['date']==i, 'b'] = popt[1]
    df_temp_coef.loc[df_temp_coef['date']==i, 'c'] = popt[2]

df_temp_coef = df_temp_coef.drop(columns=['date'])
df_temp_coef = df_temp_coef.reset_index()

In [18]:
for i in df_temp_coef['date']:
    df_daily=pd.DataFrame(index=[1,2,3], columns=['value'])
    df_daily.iloc[0,0] = df_temp_coef.loc[df_temp_coef['date']==i,'a'].iloc[0]
    df_daily.iloc[1,0] = df_temp_coef.loc[df_temp_coef['date']==i,'b'].iloc[0]
    df_daily.iloc[2,0] = df_temp_coef.loc[df_temp_coef['date']==i,'c'].iloc[0]
    
    date=pd.to_datetime(i).strftime('%Y%m%d')
    df_daily.value = df_daily.value.apply(lambda x: '%.2e' % x)
    df_daily[df_daily.index==1] = df_daily[df_daily.index==1].apply(lambda x: '%.1e' % x)
    filename = '../../data/final/box_model_input_sacramento/sacramento_temp_' + date + '_11.txt'
    print(filename)
    df_daily.to_csv(filename, index=True, header=False, sep=' ')

../../data/final/box_model_input_sacramento/sacramento_temp_20200401_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200402_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200403_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200404_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200405_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200406_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200407_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200408_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200409_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200410_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200411_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200412_11.txt
../../data/final/box_model_input_sacramento/sacramento_temp_20200413_11.txt
../../data/f